2018/9/26 Updates

Changes
1. Columns converted to proper datatypes
2. Entry and Exit numbers tracked for each turnstile in new columns 
3. df.head() in each block to show the progression of the cleaning. This will need to be removed
4. Some operations were performed because of the guidance of the challenge. 
   This is not always what we need for our project goals. I have kept the unused code in markedown cells at the end,
   incase we do need it later

Qustions:
1. Should an i.d. column be created that combines C_A, Unit, SCP, and Station? 

Next steps:
2. Fix functions
1. Calculate daily entry/exits for each day

ID explained 
Station then Unit(think enterance maybe) then CA(booth) then SCP(turnstyle)

In [1]:
import pandas as pd
import re

In [2]:
#load April - July, 2017 & 2018 MTA Turntile data
#time intensive block
df = pd.read_csv('/Users/GabeKlick/Downloads/turnstile_recordings_summer_2017_2018_clean.csv', index_col = 'Unnamed: 0')
df.head()

/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,00:00:00,REGULAR,6157740,2085315
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,04:00:00,REGULAR,6157777,2085319
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,08:00:00,REGULAR,6157810,2085353
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,12:00:00,REGULAR,6157963,2085453
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,16:00:00,REGULAR,6158212,2085529


In [8]:
#rename index column to be used later for dateframes merges
df.rename(index = {'Unnamed: 0': 'Index'}, inplace = True)
df.head()

,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,00:00:00,REGULAR,6157740,2085315
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,04:00:00,REGULAR,6157777,2085319
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,08:00:00,REGULAR,6157810,2085353
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,12:00:00,REGULAR,6157963,2085453
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,16:00:00,REGULAR,6158212,2085529


In [9]:
#exit column has a few misplaced chars preventing the column from being int values
#this block removes the chars and converts columns to int64

#apply regex statement to ensure only numbers in the column
def only_num(col):
    return re.findall('(\d)', col)
df.exits = df.exits.apply(lambda x: only_num(str(x)))

#the regex statment returns a list a numbers for each value
#join list values to single string items of numbers
df.exits = df.exits.apply(lambda x: ''.join(x))

#convert str to int64
df.exits = df.exits.apply(lambda x: int(x))

df.head()

,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,00:00:00,REGULAR,6157740,2085315
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,04:00:00,REGULAR,6157777,2085319
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,08:00:00,REGULAR,6157810,2085353
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,12:00:00,REGULAR,6157963,2085453
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,16:00:00,REGULAR,6158212,2085529


In [10]:
#combine and convert date,time column from str to datetime
#time intensive block, run then take 5

#df.date = pd.to_datetime(df.date_time, infer_datetime_format= True)
df.date = pd.to_datetime(df.date + ' ' + df.time)

#rename to showcase the new time feature in the column
df.rename({'date': 'date_time'}, axis = 'columns', inplace = True)

#removes the now redundant time column
df.drop(columns = ['time'], inplace = True)

df.head()

,c_a,unit,scp,station,linename,division,date_time,desc,entries,exits
0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29 00:00:00,REGULAR,6157740,2085315
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29 04:00:00,REGULAR,6157777,2085319
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29 08:00:00,REGULAR,6157810,2085353
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29 12:00:00,REGULAR,6157963,2085453
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29 16:00:00,REGULAR,6158212,2085529


In [16]:
#remove dates before May and after August
df = df[(df.date_time.dt.month != 4) & (df.date_time.dt.month != 8)].reset_index()
df.head()

,level_0,index,c_a,unit,scp,station,linename,division,date_time,desc,entries,exits
0,0,12,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01 00:00:00,REGULAR,6159107,2085842
1,1,13,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01 04:00:00,REGULAR,6159116,2085849
2,2,14,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01 08:00:00,REGULAR,6159166,2085976
3,3,15,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01 12:00:00,REGULAR,6159331,2086218
4,4,16,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01 16:00:00,REGULAR,6159595,2086299


In [17]:
#replace typo
df[df['c_a'] == ' "A002']['c_a'] = 'A002'

/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
difference_of_entries = df.groupby(['station', 'unit', 'c_a', 'scp'])['entries'].apply(lambda x: x- x.shift(1))
difference_of_exits = df.groupby(['station', 'unit', 'c_a', 'scp'])['exits'].apply(lambda x: x- x.shift(1))
hours_elapsed = df.groupby(['station', 'unit', 'c_a', 'scp'])['date_time'].apply(lambda x: x- x.shift(1))
temp_df = pd.DataFrame({'Index': df.index, 'num_entries': difference_of_entries, 'num_exits': difference_of_exits, 'hours_elapsed': hours_elapsed})

In [20]:
#time intensive, run then take 5
#merge the main and temp df (created in function) on the index column
df = pd.concat([df, temp_df], axis = 1, join = 'outer')
df.head()

,level_0,index,c_a,unit,scp,station,linename,division,date_time,desc,entries,exits,Index,num_entries,num_exits,hours_elapsed
0,0,12,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01 00:00:00,REGULAR,6159107,2085842,0,NaN,NaN,NaT
1,1,13,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01 04:00:00,REGULAR,6159116,2085849,1,9.0,7.0,04:00:00
2,2,14,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01 08:00:00,REGULAR,6159166,2085976,2,50.0,127.0,04:00:00
3,3,15,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01 12:00:00,REGULAR,6159331,2086218,3,165.0,242.0,04:00:00
4,4,16,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01 16:00:00,REGULAR,6159595,2086299,4,264.0,81.0,04:00:00


In [42]:
df[df['hours_elapsed'] == df['hours_elapsed'].max()]

,level_0,index,c_a,unit,scp,station,linename,division,date_time,desc,entries,exits,Index,num_entries,num_exits,hours_elapsed
1427698,1427698,1526516,N309A,R140,00-05-00,QUEENS PLAZA,EMR,IND,2018-07-21 01:00:00,REGULAR,5430,0,1427698,810.0,0.0,364 days 04:00:00
1427740,1427740,1526558,N309A,R140,00-05-01,QUEENS PLAZA,EMR,IND,2018-07-21 01:00:00,REGULAR,0,91,1427740,0.0,55.0,364 days 04:00:00
1427782,1427782,1526600,N309A,R140,00-05-02,QUEENS PLAZA,EMR,IND,2018-07-21 01:00:00,REGULAR,1095,0,1427782,143.0,0.0,364 days 04:00:00
1427824,1427824,1526642,N309A,R140,00-05-03,QUEENS PLAZA,EMR,IND,2018-07-21 01:00:00,REGULAR,0,497,1427824,0.0,33.0,364 days 04:00:00
1427866,1427866,1526684,N309A,R140,00-06-00,QUEENS PLAZA,EMR,IND,2018-07-21 01:00:00,REGULAR,8021406,930044,1427866,41760.0,151341.0,364 days 04:00:00


In [41]:
df = df[df['hours_elapsed'] != df['hours_elapsed'].max()]

In [26]:
df[df['hours_elapsed'] == df['hours_elapsed'].min()]

,level_0,index,c_a,unit,scp,station,linename,division,date_time,desc,entries,exits,Index,num_entries,num_exits,hours_elapsed
1953245,1953245,2084773,D002,R390,00-03-02,8 AV,N,BMT,2017-05-06 01:00:00,REGULAR,15777157,5811094,1953245,14571690.0,4527159.0,-452 days +04:00:00


In [27]:
df[df['num_entries'] == df['num_entries'].max()]

,level_0,index,c_a,unit,scp,station,linename,division,date_time,desc,entries,exits,Index,num_entries,num_exits,hours_elapsed
1864221,1864221,1995749,A061,R142,00-00-02,DELANCEY/ESSEX,FJMZ,BMT,2018-06-09 01:00:00,REGULAR,1646619071,1057689610,1864221,1.646614e+09,1.057688e+09,-42 days +04:00:00


In [28]:
df[df['num_entries'] == df['num_entries'].min()]

,level_0,index,c_a,unit,scp,station,linename,division,date_time,desc,entries,exits,Index,num_entries,num_exits,hours_elapsed
1787835,1787835,1919363,A061,R142,00-00-02,DELANCEY/ESSEX,FJMZ,BMT,2018-07-18 13:00:00,RECOVR AUD,7,0,1787835,-1.646681e+09,-1.057707e+09,2 days 04:00:00


In [29]:
df[df['num_exits'] == df['num_exits'].max()]

,level_0,index,c_a,unit,scp,station,linename,division,date_time,desc,entries,exits,Index,num_entries,num_exits,hours_elapsed
1346644,1346644,1445462,N215,R237,00-00-02,182-183 STS,BD,IND,2018-06-30,REGULAR,1493111155,1845552454,1346644,1.482180e+09,1.838317e+09,28 days 04:00:00


In [30]:
df[df['num_exits'] == df['num_exits'].min()]

,level_0,index,c_a,unit,scp,station,linename,division,date_time,desc,entries,exits,Index,num_entries,num_exits,hours_elapsed
1543682,1543682,1675210,N215,R237,00-00-02,182-183 STS,BD,IND,2018-05-19,REGULAR,10911801,7230222,1543682,-1.482157e+09,-1.838335e+09,-74 days +04:00:00


Unused Code Below

#Entry and Exits data recorded as a 'snapshot' every 4 hours
#Adding columns tracking passanger entery and exit evert 4 hours
df = df.assign(entry_count = df['entries'] - df['entries'].shift(1))
df = df.assign(exit_count = df['exits'] - df['exits'].shift(1))

This block can be used for the challenge but not neccessary for our goals for the project
returns values in the formate requested by the challages 

#This blocks combines the date and time columns so they may later be converted to a datetime type

#combines columns, separated by a space, into the pre-exisiting column 'date'
df.date = df.date + ' '+ df.time

#rename to showcase the new time feature in the column
df.rename({'date': 'date_time'}, axis = 'columns', inplace = True)

#removes the now redundant time column
df.drop(columns = ['time'], inplace= True)

#converting date_time from string to datetime
df.date_time = df.date_time.apply(lambda x: dateutil.parser.parse(x))


#time intensive. Run, then take 5
# df.date = df.date.apply(lambda x: dateutil.parser.parse(x))

Again, may be used for the challenge, but not necessary for the project

#create a new column that combines the date and time columns, separated by a single space
df['date_time'] = df.date + ' '+ df.time
#Warning: time intensive block, run then take 5
#converting date_time from string to datetime
df.date_time = df.date_time.apply(lambda x: dateutil.parser.parse(x))
df.head()
#currently returning datetime in same formate it is in: mm/dd/yyyy hh:mm:ss
#different than the challenge scope, but seems 

Incase any issues arrive with exit_entry function, this is the originial code

#This should be a function. See above
#time intensive. Run, then take 5

#Groups turnstiles by station, turnstyle id, and date
#within each group find the difference between the consecutive entry points 
#this difference is stored as a series
df.test = df.groupby(['station','scp','date']).entries.apply(lambda x: x - x.shift(1))

#create a temp df that uses the same index column as the main df

#merge the main and temp df on the index column
df = pd.merge(df, df_entry, how = 'outer', on = 'Index')

df.head(10)

This function is no longer being used

'''
Creates a new series that groups turnstyles by station, id, and date, 
and find the difference in consecutive data points
stores this difference as a series
crease a temp df using difference and the main df's index column
'''
def entry_exit_calc(column, new_col_name, index_name):
    #Groups turnstiles by station, turnstyle id, and date
    #within each group find the difference between the consecutive entry points 
    #this difference is stored as a series
    difference_of_entries = df.groupby(['station', 'unit', 'c_a', 'scp'])[column].apply(lambda x: x- x.shift(1))
    
    hours_elapsed = df.groupby(['station', 'unit', 'c_a', 'scp'])['date_time'].apply(lambda x: x- x.shift(1)) #make sure date_time is correct name
    #create a temp df that uses the same index column as the main df
    return pd.DataFrame({'Index': df[index_name], new_col_name: difference_of_tolls, hours_elapsed: hours_elapsed})
    